In [19]:
import os
import requests
import pandas as pd
from openai import OpenAI
from datetime import datetime
# from twilio.rest import Client
import smtplib
from email.mime.text import MIMEText

from dotenv import load_dotenv

load_dotenv()

True

In [4]:
def refresh_access_token():
    res = requests.post(
        "https://www.strava.com/oauth/token",
        data={
            "client_id": os.getenv("STRAVA_CLIENT_ID"),
            "client_secret": os.getenv("STRAVA_CLIENT_SECRET"),
            "refresh_token": os.getenv("STRAVA_REFRESH_TOKEN"),
            "grant_type": "refresh_token",
        }
    )
    print(res.status_code)
    print(res.json())
    res.raise_for_status()
    return res.json()["access_token"]

def get_strava_activities(access_token, per_page=5):
    url = "https://www.strava.com/api/v3/athlete/activities"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"per_page": per_page}
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

In [5]:
# # get strava access token
# token_data = refresh_access_token()
# # call up strava
# strava_data = get_strava_activities(access_token=token_data, per_page=10)


In [6]:
# one function to get token, call strava, and return formatted dataframe with latest events

def call_strava_and_format_response(activity_count=10):
    """this gets a refreshed strava access token, calls strava to get latest n activities,
    formats them with better units, and drops unnecessary columns. It returns a pd df wih 10 rows in
    descending order by date"""
    # get strava access token
    token_data = refresh_access_token()
    # call up strava
    strava_data = get_strava_activities(access_token=token_data, per_page=activity_count)

    strava_results = pd.DataFrame(strava_data)

    strava_results['moving_time_mins'] = round(strava_results['moving_time']/60, 1)
    strava_results['distance_miles'] = round(strava_results['distance']/1609.34, 2)
    strava_results['elevation_gain_ft'] = round(strava_results['total_elevation_gain']*3.281, 1)

    filtered_strava_results = strava_results[['distance_miles', 'moving_time_mins', 'elevation_gain_ft', 'type', 'start_date_local', 'average_speed', 'average_heartrate', 'max_heartrate']]


    return filtered_strava_results


In [7]:
call_strava_and_format_response(activity_count=10)

200
{'token_type': 'Bearer', 'access_token': '4f753b27c4216883055dd2ed091a62032add4821', 'expires_at': 1748989269, 'expires_in': 21026, 'refresh_token': 'c280cb6536c00d9cbcd05b660423cb5ee91e34c8'}


,distance_miles,moving_time_mins,elevation_gain_ft,type,start_date_local,average_speed,average_heartrate,max_heartrate
0,5.01,58.2,138.5,Run,2025-06-01T13:08:00Z,2.308,159.2,180.0
1,3.02,40.5,112.9,Run,2025-05-28T17:35:23Z,1.995,145.0,158.0
2,8.53,97.3,369.4,Run,2025-05-26T11:52:59Z,2.352,169.3,187.0
3,4.99,55.8,0.0,Run,2025-05-23T09:14:03Z,2.394,161.9,180.0
4,6.01,68.7,168.0,Run,2025-05-20T07:30:00Z,2.347,163.2,180.0
5,4.12,92.2,874.1,Walk,2025-05-19T19:45:26Z,1.200,102.8,161.0
6,5.50,73.8,232.3,Run,2025-05-18T17:34:04Z,1.999,149.7,170.0
7,4.51,52.8,34.8,Run,2025-05-16T10:10:11Z,2.293,174.4,192.0
8,2.55,29.1,207.0,Run,2025-05-13T12:47:05Z,2.354,163.2,182.0
9,6.04,69.5,207.0,Run,2025-05-11T12:53:14Z,2.334,169.3,189.0


In [8]:
filtered_strava_results = call_strava_and_format_response(activity_count=10)

200
{'token_type': 'Bearer', 'access_token': '4f753b27c4216883055dd2ed091a62032add4821', 'expires_at': 1748989269, 'expires_in': 21024, 'refresh_token': 'c280cb6536c00d9cbcd05b660423cb5ee91e34c8'}


In [24]:
# create a prompt requesting an encouraging good morning message based on recent strava data

def create_good_morning_strava_prompt():
    """this returns a PROMPT to request a good morning message that includes today's date and
    an encouraging message based on the last 10 strava activities"""
    # get latest 10 strava activities (returns a pd df)
    filtered_strava_results = call_strava_and_format_response(activity_count=10)

    # sort activities just in case
    df = filtered_strava_results.sort_values("start_date_local", ascending=False)

    # Format each activity for the prompt
    activity_lines = []
    for _, row in df.iterrows():
        date = pd.to_datetime(row['start_date_local']).strftime('%Y-%m-%d')
        activity_type = row['type']
        distance = f"{row['distance_miles']:.2f} mi"
        time = f"{row['moving_time_mins']:.1f} min"
        elev = f"{row['elevation_gain_ft']:.0f} ft"
        avg_hr = f"{row['average_heartrate']:.0f}" if not pd.isnull(row['average_heartrate']) else "N/A"
        max_hr = f"{row['max_heartrate']:.0f}" if not pd.isnull(row['max_heartrate']) else "N/A"
        line = f"{date}: {activity_type} - {distance}, {time}, {elev} gain, Avg HR: {avg_hr}, Max HR: {max_hr}"
        activity_lines.append(line)

    # add date
    today = datetime.now().strftime("%Y-%m-%d")

    # Create the prompt
    activity_str = "\n".join(activity_lines)
    prompt = (
        f"Today is {today}.\n"
        "Here are my most recent Strava activities:\n"
        f"{activity_str}\n\n"
        """Please send me a good morning text message that includes an encouraging message about my recent strava activity and incorporates today's date.
        for example, you could tell me how my mileage this week compares to previous weeks or tell me if i've had a particularly fast run recently."""
    )

    return(prompt)

In [25]:
def call_large_model(instructions, prompt):
    client = OpenAI()

    # NOTES ON PROMPT ENGINEERING
    # -- If you say it's an expert on history of SF as well as local flora/fauna and then ask for a fact, every fact will be about GGP being bigger than central park
    # -- chatGPT seems to hallucinate a lot more via API than in the GUI even for the exact same model/prompt.

    response = client.responses.create(
        model="gpt-4.1",
        instructions = instructions,
        input = prompt,
    )

    return(response.output_text)

In [ ]:

# create function to send SMS message from my email (free) instead of using twilio
def send_text_from_email(message_body="nobody told me what to say!"):
    """takes a string and sends SMS from my email address"""
    to_number = os.getenv('MY_CELL_EMAIL')
    from_email = os.getenv("MY_EMAIL")
    app_password = os.getenv("GMAIL_APP_PASSWORD")

    # create message
    msg = MIMEText(message_body)
    msg['From'] = from_email
    msg['To'] = to_number

    # dale
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(from_email, app_password)
        server.sendmail(from_email, [to_number], msg.as_string())

    print("Sent!")

In [ ]:
instructions = """You are a supportive life coach and personal trainer named Zeus Groos. Your trainee, Zach, is training for the New York Marathon on November 3 2025.
            He is doing a half marathon training program that ends on June 29 and then he'll begin the full marathon training program. You should play the role of 
            an expert coach, giving general running advice as well as specific feedback tailored to your trainee's workout history. This will be sent via SMS - please format accordingly."""

prompt = create_good_morning_strava_prompt()

message_body = call_large_model(instructions=instructions, prompt=prompt)

send_text_from_email(message_body=message_body)

Sent!


In [ ]:
# # Load environment variables from .env file
# load_dotenv()

# account_sid = os.getenv("TWILIO_ACCOUNT_SID")
# auth_token = os.getenv("TWILIO_AUTH_TOKEN")
# twilio_number = os.getenv("TWILIO_PHONE_NUMBER")
# my_cell = os.getenv("MY_CELL_NUMBER")
# # my_cell = "+17147560044"

# twilio_client = Client(account_sid, auth_token)

# message = twilio_client.messages.create(
#     body="Sent from your Twilio trial account - Hello from Twilio!",
#     from_=twilio_number,
#     to=my_cell
# )

# print(f"Message sent! SID: {message.sid}")
# print(f"Message status: {message.status}")

Message sent! SID: SMe46f871f3aea24ef7b0d4527567464fb
Message status: queued


In [36]:
send_text_from_email()

Sent!
